MNIST Computational graph development

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)
x_input = tf.placeholder(dtype=tf.float32, shape=[None, 784], name='x_input')
W = tf.Variable(initial_value=tf.zeros(shape=[784,10]), name='W')
b = tf.Variable(initial_value=tf.zeros(shape=[10]), name='b')
y_actual = tf.add(x=tf.matmul(a=x_input, b=W, name='matmul'), y=b, name='y_actual')
y_expected = tf.placeholder(dtype=tf.float32, shape=[None,10], name='y_expected')
cross_entropy_loss = tf.reduce_mean(input_tensor=tf.nn.softmax_cross_entropy_with_logits(labels=y_expected, logits=y_actual), name='cross_entropy_loss')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5, name='optimizer')
train_step = optimizer.minimize(loss=cross_entropy_loss, name='train_step')
saver = tf.train.Saver()
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())
tf.train.write_graph(graph_or_graph_def=session.graph_def,
                     logdir='.',
                     name='mnist_model.pbtxt',
                     as_text=False)
for _ in range(1000):
  batch = mnist_data.train.next_batch(100)
  train_step.run(feed_dict={x_input: batch[0], y_expected: batch[1]})
saver.save(sess=session,
           save_path='mnist_model.ckpt')

correct_prediction = tf.equal(x=tf.argmax(y_actual, 1),y=tf.argmax(y_expected,1))
accuracy = tf.reduce_mean(tf.cast(x=correct_prediction, dtype=tf.float32))
print(accuracy.eval(feed_dict={x_input: mnist_data.test.images, y_expected: mnist_data.test.labels}))
print(session.run(fetches=y_actual, feed_dict={x_input: [mnist_data.test.images[0]]}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


0.9195
[[ 0.27319932 -8.558873    0.9127517   4.431087   -2.9133983  -0.23717833
  -7.781933   11.06955     0.05514276  2.749658  ]]


Freeze graph

In [3]:
import tensorflow as tf
from tensorflow.python.tools import freeze_graph, optimize_for_inference_lib
freeze_graph.freeze_graph(input_graph='mnist_model.pbtxt',
                          input_saver='',
                          input_binary=True,
                          input_checkpoint='mnist_model.ckpt',
                          output_node_names='y_actual',
                          restore_op_name='save/restore_all',
                          filename_tensor_name='save/Const:0',
                          output_graph='frozen_mnist_model.pb',
                          clear_devices=True,
                          initializer_nodes='')
input_graph_def = tf.GraphDef()
with tf.gfile.Open('frozen_mnist_model.pb', 'rb') as f:
  data = f.read()
  input_graph_def.ParseFromString(data)
output_graph_def = optimize_for_inference_lib.optimize_for_inference(input_graph_def=input_graph_def,
                                                                     input_node_names=['x_input'],
                                                                     output_node_names=['y_actual'],
                                                                     placeholder_type_enum=tf.float32.as_datatype_enum)
f = tf.gfile.FastGFile(name='optimized_frozen_mnist_model.pb',
                       mode='w')
f.write(file_content=output_graph_def.SerializeToString())

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from mnist_model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
Instructions for updating:
Use `tf.compat.v1.graph_util.remove_training_nodes`
Instructions for updating:
Use tf.gfile.GFile.
